# Superstore Sales Prediction

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score



In [3]:
sales = pd.read_csv('data/train.csv')

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

Dataset has 9800 rows and there are null values in 'Postal Code' column

In [6]:
sales.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


# Data Pre-processing

In [17]:
sales.Country.unique()

array(['United States'], dtype=object)

**Columns such as 'Country' have a single value and can be dropped from dataset. Likewise, 'Row ID' is a column that won't affect the data analysis and can be dropped. We do not need Product or Customer specific information, so we can drop these columns.**

In [21]:
def process_data(df):
    df = df.copy()
    
    # Drop columns that won't affect data analysis
    df = df.drop(['Row ID','Country', 'Customer ID', 'Customer Name', 'Order ID', 'Product Name'], axis=1)
    
    return df

In [22]:
X = process_data(sales)

In [24]:
X

,Order Date,Ship Date,Ship Mode,Segment,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Sales
0,08/11/2017,11/11/2017,Second Class,Consumer,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,261.9600
1,08/11/2017,11/11/2017,Second Class,Consumer,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,731.9400
2,12/06/2017,16/06/2017,Second Class,Corporate,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,14.6200
3,11/10/2016,18/10/2016,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,957.5775
4,11/10/2016,18/10/2016,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...
9795,21/05/2017,28/05/2017,Standard Class,Corporate,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,3.7980
9796,12/01/2016,17/01/2016,Standard Class,Corporate,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,10.3680
9797,12/01/2016,17/01/2016,Standard Class,Corporate,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,235.1880
9798,12/01/2016,17/01/2016,Standard Class,Corporate,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,26.3760


**Process date columns**

In [ ]:
from datetime import datetime

# Convert columns to datetime type
X['Order Date'] = pd.to_datetime(X['Order Date'])
X['Ship Date'] = pd.to_datetime(X['Ship Date'])
X

In [41]:
# Extract year and month
X['Year_Order'] = pd.DatetimeIndex(X['Order Date']).year
X['Month_Order'] = pd.DatetimeIndex(X['Order Date']).month

X['Year_Ship'] = pd.DatetimeIndex(X['Ship Date']).year
X['Month_Ship'] = pd.DatetimeIndex(X['Ship Date']).month

# Drop original columns
X.drop(['Order Date','Ship Date'], axis = 1, inplace=True)
X

,Ship Mode,Segment,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Sales,Month_Order,Year_Order,Year_Ship,Month_Ship
0,Second Class,Consumer,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,261.9600,8,2017,2017,11
1,Second Class,Consumer,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,731.9400,8,2017,2017,11
2,Second Class,Corporate,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,14.6200,12,2017,2017,6
3,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,957.5775,11,2016,2016,10
4,Standard Class,Consumer,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,22.3680,11,2016,2016,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,Standard Class,Corporate,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,3.7980,5,2017,2017,5
9796,Standard Class,Corporate,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,10.3680,12,2016,2016,1
9797,Standard Class,Corporate,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,235.1880,12,2016,2016,1
9798,Standard Class,Corporate,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,26.3760,12,2016,2016,1


**Deal with categorical column values**

In [71]:
# List columns and the number of unique values in each column
{column: X[column].nunique() for column in X.columns}

{'Ship Mode': 4,
 'Segment': 3,
 'City': 529,
 'State': 49,
 'Postal Code': 626,
 'Region': 4,
 'Product ID': 1861,
 'Category': 3,
 'Sub-Category': 17,
 'Sales': 5757,
 'Month_Order': 12,
 'Year_Order': 4,
 'Year_Ship': 5,
 'Month_Ship': 12}